In [1]:
import os 

os.chdir("..")

import numpy as np
from tslearn.generators import random_walks
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tscluster.opttscluster import OptTSCluster
from tscluster.tskmeans import TSKmeans, TSGlobalKmeans
from tscluster.preprocessing import utils as preprocess_utils, TSStandardScaler, TSMinMaxScaler
from tscluster.metrics import inertia, max_dist

In [2]:
os.chdir("./test/sample_data")

In [3]:
X_mini = np.load("./sythetic_data.npy")

In [4]:
kc_opt_model = OptTSCluster(3, random_state=42, use_full_constraints=1, warm_start=1, 
                          scheme='z1c1', n_allow_assignment_change=None,
                             use_MILP_centroid=True, is_tight_constraints=False, is_Z_positive=True,
                             init_with_prev=False, use_sum_distance=False)

kc_opt_model.fit(X_mini, verbose=True)

Warm starting...
Done with warm start after 0.11secs

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-18

100.0% of data used
Done with 1 of 1. Ehat: [1.52196375], Estar: [1.52196375]

Total time is 0.52secs



In [5]:
print(f"inertia score is {inertia('./sythetic_data.npy', kc_opt_model.cluster_centers_, kc_opt_model.labels_, ord=1)}")
print(f"max_dist score is {max_dist('./sythetic_data.npy', kc_opt_model.cluster_centers_, kc_opt_model.labels_, ord=1)}")
print(f"shape of labels_ is {kc_opt_model.labels_.shape}")
print("head of labels_ is:")
print(kc_opt_model.labels_[:5, :])

print(f"shape of cluster_centers_ is {kc_opt_model.cluster_centers_.shape}")
print("head of cluster_centers_ is:")

if len(kc_opt_model.cluster_centers_.shape) == 3:
    print(kc_opt_model.cluster_centers_[:5, :, :])
else:
    print(kc_opt_model.cluster_centers_)

inertia score is 214.40647069697624
max_dist score is 1.5219637544500682
shape of labels_ is (30, 10)
head of labels_ is:
[[2 2 2 2 2 2 2 2 2 2]
 [1 1 1 1 1 1 1 1 1 1]
 [1 2 1 1 1 1 1 1 1 2]
 [2 0 0 2 0 0 0 0 0 0]
 [1 2 0 1 1 1 1 1 1 2]]
shape of cluster_centers_ is (10, 3, 1)
head of cluster_centers_ is:
[[[ 8.3640779 ]
  [15.38092986]
  [ 9.87721949]]

 [[11.50046295]
  [10.04978372]
  [ 9.40500694]]

 [[ 9.45626162]
  [ 9.41879279]
  [10.36034458]]

 [[10.68152685]
  [ 6.15811016]
  [10.68152685]]

 [[ 8.58872533]
  [ 5.36710865]
  [12.27555051]]]


In [6]:
kc_opt_model.get_named_labels(
    time=['t'+str(t) for t in range(kc_opt_model.fitted_data_shape_[0])], 
    entities=['e'+str(e) for e in range(kc_opt_model.fitted_data_shape_[1])]
).head()

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
e0,2,2,2,2,2,2,2,2,2,2
e1,1,1,1,1,1,1,1,1,1,1
e2,1,2,1,1,1,1,1,1,1,2
e3,2,0,0,2,0,0,0,0,0,0
e4,1,2,0,1,1,1,1,1,1,2


In [7]:
centers = kc_opt_model.get_named_cluster_centers(
    time=['t'+str(t) for t in range(kc_opt_model.fitted_data_shape_[0])], 
    features=['f'+str(f) for f in range(kc_opt_model.fitted_data_shape_[2])]
)
len(centers)

3

In [8]:
centers[0]

,f0
t0,8.364078
t1,11.500463
t2,9.456262
t3,10.681527
t4,8.588725
t5,11.339732
t6,8.403033
t7,11.649322
t8,8.134154
t9,12.781047


In [9]:
centers[1]

,f0
t0,15.380930
t1,10.049784
t2,9.418793
t3,6.158110
t4,5.367109
t5,5.464376
t6,6.618689
t7,9.106237
t8,11.315930
t9,13.775484


In [ ]:
X = random_walks(n_ts=50, sz=32, d=2, random_state=42)
Xt = preprocess_utils.ntf_to_tnf(X)

In [ ]:
km = TSKmeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0)
km.fit(X, arr_format="NTF")
print(km.Xt.shape)
print(km.labels_)
print(len(km.labels_))
print(km.cluster_centers_.shape)
print(km.cluster_centers_[0, :5, :])
print()

In [ ]:
km.get_named_labels(time=['t'+str(t) for t in range(km.fitted_data_shape_[0])], entities=['e'+str(e) for e in range(km.fitted_data_shape_[1])]).head()

In [ ]:
km2 = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0).fit(X)
print(km2.labels_)
print(len(km2.labels_))
print(km2.cluster_centers_[:5, 0, :])

In [ ]:
km3 = TSGlobalKmeans(n_clusters=3)
km3.fit(Xt)
print(km3.labels_)
print(km3.labels_.shape)

In [ ]:
km3.get_named_labels(
    time=['t'+str(t) for t in range(km3.fitted_data_shape_[0])], 
    entities=['e'+str(e) for e in range(km3.fitted_data_shape_[1])]
).head()

In [ ]:
centers = km3.get_named_cluster_centers(
    time=['t'+str(t) for t in range(km3.fitted_data_shape_[0])], 
    features=['f'+str(f) for f in range(km3.fitted_data_shape_[2])]
)
len(centers)

In [ ]:
centers[0].head()

In [ ]:
print("Z score")

print("per_time =True")

scaler = TSStandardScaler(per_time=True)
scaler.fit(X, arr_format='NTF')
X_scaled = scaler.transform(X, arr_format='NTF',) 
print(f"Zscore X_scaled shape is {X_scaled.shape}")
print(X_scaled[0, :5, :])
print(f"Original data is:")
print(Xt[0, :5, :])
print("Inverse is: ")
print(scaler.inverse_transform(X_scaled)[0, :5, :])
print()

In [ ]:
sk_scaler = StandardScaler()
print(sk_scaler.fit_transform(Xt[0])[:5])

print("per_time = False")

scaler = TSStandardScaler(per_time=False)
X_scaled = scaler.fit_transform(Xt)
print(f"Zscore X_scaled shape is {X_scaled.shape}")
print(X_scaled[0, :5, :])
print(f"Original data is:")
print(Xt[0, :5, :])
print("Inverse is: ")
print(scaler.inverse_transform(X_scaled)[0, :5, :])
print()

In [ ]:
sk_scaler = StandardScaler()
print(sk_scaler.fit_transform(np.vstack(Xt))[:5])


print("MinMax")

print("per_time =True")

scaler = TSMinMaxScaler(per_time=True)
X_scaled = scaler.fit_transform(Xt)
print(f"MinMax X_scaled shape is {X_scaled.shape}")
print(X_scaled[0, :5, :])
print(f"Original data is:")
print(Xt[0, :5, :])
print("Inverse is: ")
print(scaler.inverse_transform(X_scaled)[0, :5, :])
print()

In [ ]:
sk_scaler = MinMaxScaler()
print(sk_scaler.fit_transform(Xt[0])[:5])

print("per_time = False")

scaler = TSMinMaxScaler(per_time=False)
X_scaled = scaler.fit_transform(Xt)
print(f"MinMax X_scaled shape is {X_scaled.shape}")
print(X_scaled[0, :5, :])
print(f"Original data is:")
print(Xt[0, :5, :])
print("Inverse is: ")
print(scaler.inverse_transform(X_scaled)[0, :5, :])
print()

In [ ]:
sk_scaler = MinMaxScaler()
print(sk_scaler.fit_transform(np.vstack(Xt))[:5])

scaler = TSMinMaxScaler(per_time=False)
print()
print("trying fit(str) for csv")
Xt = scaler.fit_transform("./synthetic_csv")
print(Xt[0, :5, :])
print("Inverse tranform for csv")
print(scaler.inverse_transform(Xt)[0, :5, :])

In [ ]:
print()
print("trying fit(str) for json")
Xt = scaler.fit_transform("./synthetic_json")
print(Xt[0, :5, :])
print("Inverse tranform for json")
print(scaler.inverse_transform(Xt)[0, :5, :])

In [ ]:
print()
print("trying fit(str) for npy")
Xt = scaler.fit_transform("./synthetic_npy")
print(Xt[0, :5, :])
print("Inverse tranform for npy")
print(scaler.inverse_transform(Xt)[0, :5, :])

In [ ]:
print()
print("trying fit(lst) for csv")
file_list = [
    "./synthetic_csv/timestep_0.csv",
    "./synthetic_csv/timestep_1.csv",
    "./synthetic_csv/timestep_2.csv",
    "./synthetic_csv/timestep_3.csv",
    "./synthetic_csv/timestep_4.csv"
]

In [ ]:
scaler.fit(file_list, read_file_args={'header': 0, 'sep': ","})
Xt = scaler.transform(file_list, read_file_args={'header': None, 'sep': ","})
print(Xt[0, :5, :])
print("Inverse tranform for csv")
print(scaler.inverse_transform(Xt)[0, :5, :])

In [ ]:
print()
print("fit_transform")
# scaler.fit(file_list, read_file_args={'header': None, 'sep': ","})
Xt = scaler.fit_transform(file_list, read_file_args={'header': None, 'sep': ","})
print(Xt[0, :5, :])
print("Inverse tranform for csv")
print(scaler.inverse_transform(Xt)[0, :5, :])